# Importing python libraries

In [1]:
import gensim
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim import similarities
from nltk.tokenize import WordPunctTokenizer 
tk = WordPunctTokenizer() 

In [2]:
def preprocessing_text(input_text):
#     print(input_text)
    input_words_list = tk.tokenize(input_text)
#     print(input_words_list)
    clean_input_words_list = []
    for word in input_words_list:
        word = word.lower()
        result = list(filter(lambda x: x.isalpha(), word))
        word = ''.join(result)
#         print(result)
#         print(word)
        if word not in clean_input_words_list and len(word) > 2 :
            clean_input_words_list.append(word)
#     print(clean_input_words_list)
    return clean_input_words_list
            

# Creating Index
- Index is nothing but fast lookup of a query

In [3]:
def create_index(text_corpus) :

    frequency = defaultdict(int)
    for text in text_corpus:
        for token in text:
            frequency[token] += 1

    # Only keep words that appear more than once
    processed_corpus = [[token for token in text if frequency[token] > 1] for text in text_corpus]
    
    # creating dictonary from corpus
    dictionary = corpora.Dictionary(processed_corpus)
    
    # creating bow
    bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

    # train the model
    tfidf = models.TfidfModel(bow_corpus)

    index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=len(bow_corpus), num_best=10)
    return index, dictionary, tfidf

# Creating Index for GCPH_BRAND_HIERARCHY 

In [4]:
df_brand = pd.read_csv('allskus_gcph_brand.csv')
df_brand = df_brand[df_brand.region=='CA']
df_brand['brand_desc'] = df_brand['regional_brand']+' '+df_brand['description']
df_brand.dropna(subset=['brand_desc'],axis=0, inplace=True)
df_brand['clean_brand_desc'] = df_brand['brand_desc'].apply(lambda x: preprocessing_text(x))
text_corpus_brand = df_brand['clean_brand_desc'].tolist()
index_brand, dictionary_brand, tfidf_brand = create_index(text_corpus_brand)
# index_brand.save('./model/index_brand')
# dictionary_brand.save('./model/dictionary_brand')
# tfidf_brand.save('./model/tfidf_brand')

d:\Users\AGouda2\python project folder\tamr-data-layer\venv\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
# dictionary_brand = gensim.corpora.Dictionary.load('./model/dictionary_brand')
# tfidf_brand = gensim.models.TfidfModel.load('./model/tfidf_brand')
# index_brand = similarities.SparseMatrixSimilarity.load('./model/index_brand')


In [19]:
# brand_path =  df_brand['finalClassificationPath']
# brand_list = brand_path.str.split("|")
# brand_list_1 = {item[1].lower() for item in brand_list}
# for item in brand_list_1:
#     print(item)

# Creating Index for GCPH_CATEGORY_HIERARCHY 

In [5]:
df_cat = pd.read_csv('allskus_gcph_category.csv')
df_cat = df_cat[df_cat.region=='CA']
df_cat['brand_desc'] = df_cat['regional_brand']+' '+df_cat['description']
df_cat.dropna(subset=['brand_desc'],axis=0, inplace=True)
df_cat['clean_cat_desc'] = df_cat['brand_desc'].apply(lambda x: preprocessing_text(x))
text_corpus_cat = df_cat['clean_cat_desc'].tolist()
index_cat, dictionary_cat, tfidf_cat = create_index(text_corpus_cat)
# index_cat.save('./model/index_cat')
# dictionary_cat.save('./model/dictionary_cat')
# tfidf_cat.save('./model/tfidf_cat')

In [9]:
# index_cat.save('./model/index_cat')
# dictionary_cat.save('./model/dictionary_cat')
# tfidf_cat.save('./model/tfidf_cat')

In [10]:
# dictionary_cat = gensim.corpora.Dictionary.load('./model/dictionary_cat')
# tfidf_cat = gensim.models.TfidfModel.load('./model/tfidf_cat')
# index_cat = similarities.SparseMatrixSimilarity.load('./model/index_cat')

# Formating input query
- Removing numbers like 50ml , 1gm, SPF70
- lower case all text

In [6]:
def get_similar_doc(query_document, tfidf, dictionary, index):
    
    query_bow = dictionary.doc2bow(query_document)
    sims = index[tfidf[query_bow]]
    return sims

# Getting no_match data from input file

In [7]:
df2 = pd.read_csv('tamr_formatted_output_ca_20200713.csv')
df2 = df2[df2.match_type=='no_match']
df2['brand_desc'] = df2['page_title']+' '+df2['brand']
query_list = df2.brand_desc.tolist()

In [8]:
df2.count()

brand               505
market              505
page_title          505
page_url            505
product_name        505
sku                 328
match_type          505
material_number       0
ean_upc               0
material_name         0
gcph_brand          505
region              505
gcph_subbrand       505
gcph_varient        505
gcph_needstate      505
gcph_category       505
gcph_subcategory    505
gcph_segment        505
gcph_subsegment     505
brand_desc          505
dtype: int64

# Starting lookup for all no match page title field

In [10]:
final_output = []
for query in query_list:
    print("Query is ==== >", query)
    query_document = preprocessing_text(query)
    sims_brand = get_similar_doc(query_document, tfidf_brand, dictionary_brand, index_brand)
    sims_cat = get_similar_doc(query_document, tfidf_cat, dictionary_cat, index_cat)
    
    for document_number, score in sorted(enumerate(sims_brand), key=lambda x: x[1][1], reverse=True)[:1]:
        print(document_number)
        print(score)
        document_number = score[0]
        hierarchy_dict = {}
        if score[1] > 0.1:
            print("Match in Brand Sheet is : ", text_corpus_brand[document_number], "Score is : ", score[1])
            brand_path =  df_brand.iloc[document_number]['finalClassificationPath']
            brand_list = brand_path.split("|")
            hierarchy_dict.update({"gfo": brand_list[0], "gcph_brand": brand_list[1], "gcph_subbrand": brand_list[2], "gcph_varient": brand_list[3], "python_score": score[1]})
        else:
            print("Not Found")
            
    for document_number, score in sorted(enumerate(sims_cat), key=lambda x: x[1][1], reverse=True)[:1]:
        document_number = score[0]
        if score[1] > 0.1:
            print("Match in Category sheet is : ", text_corpus_cat[document_number], "Score is : ", score[1])
            cat_path =  df_cat.iloc[document_number]['finalClassificationPath']
            cat_list = cat_path.split("|")
            hierarchy_dict.update({"gcph_needstate": cat_list[1], "gcph_category": cat_list[2], "gcph_subcategory": cat_list[3], "gcph_segment": cat_list[4],
                                  "gcph_subsegment" : cat_list[5]})
        else:
            print("Not Found")
    for key, val in hierarchy_dict.items():
        print(key, " : ", val)
#     final_output.append(hierarchy_dict)
#     print("\n\n\n")

Query is ==== > Sirop BENYLIN pour enfants Rhume et toux  benylin
0
(3640, 1.0)
Match in Brand Sheet is :  ['benylin', 'stfy', 'nos', 'infdrpsml'] Score is :  1.0
Match in Category sheet is :  ['benylin', 'adults'] Score is :  1.0
gfo  :  Self Care
gcph_brand  :  Benylin
gcph_subbrand  :  MS Cold
gcph_varient  :  MS Cold
python_score  :  1.0
gcph_needstate  :  Cold & Cough
gcph_category  :  Cold & Cough
gcph_subcategory  :  CCFS
gcph_segment  :  Cough
gcph_subsegment  :  Cough
Query is ==== > NETTOYANT MOUSSANT AVEENO ULTRA-CALMING aveeno2
0
(5081, 0.8735015988349915)
Match in Brand Sheet is :  ['aveeno', 'ave', 'ultra', 'calming', 'dailymoist', 'spf'] Score is :  0.8735015988349915
Match in Category sheet is :  ['aveeno', 'ave', 'ultra', 'calming', 'dailymoist', 'spf'] Score is :  0.8735015988349915
gfo  :  Skin Health
gcph_brand  :  Aveeno
gcph_subbrand  :  Ultra Calming
gcph_varient  :  Ultra Calming
python_score  :  0.8735015988349915
gcph_needstate  :  Body Care
gcph_category  :  

gcph_category  :  Wound Protection
gcph_subcategory  :  Adhesive Bandages
gcph_segment  :  Adhesive Bandages
gcph_subsegment  :  Adhesive Bandages
Query is ==== > Nettoyant antiseptique BAND-AID SANSDOULEUR, 177 ml  bandaid
0
(2911, 0.9006738662719727)
Match in Brand Sheet is :  ['band', 'aid', 'brand', 'bandaid', 'baby', 'shark'] Score is :  0.9006738662719727
Match in Category sheet is :  ['band', 'aid', 'brand', 'bandaid', 'baby', 'shark'] Score is :  0.9006738662719727
gfo  :  Essential Health
gcph_brand  :  Band Aid
gcph_subbrand  :  Deco, Decorative, Decorated
gcph_varient  :  Deco, Decorative, Decorated
python_score  :  0.9006738662719727
gcph_needstate  :  Wound Care
gcph_category  :  Wound Protection
gcph_subcategory  :  Adhesive Bandages
gcph_segment  :  Adhesive Bandages
gcph_subsegment  :  Adhesive Bandages
Query is ==== > Pansements adhésifs assortis BAND-AID Marvel Spider-Man, 20 u.  bandaid
0
(2911, 0.6488240957260132)
Match in Brand Sheet is :  ['band', 'aid', 'brand', 

gfo  :  Skin Health
gcph_brand  :  Aveeno
gcph_subbrand  :  Daily Moisturizing
gcph_varient  :  Daily Moisturizing - Core
python_score  :  0.9325114488601685
gcph_needstate  :  Baby Care
gcph_category  :  Baby Toiletries
gcph_subcategory  :  Baby Body Care
gcph_segment  :  Baby Milks/Lotions
gcph_subsegment  :  Baby Milks/Lotions
Query is ==== > Caplets MOTRIN Platine Muscles et corps motrin
0
(6506, 0.8035695552825928)
Match in Brand Sheet is :  ['tylenol', 'caplets', 'xtrayc'] Score is :  0.8035695552825928
Match in Category sheet is :  ['tylenol', 'caplets', 'xtrayc'] Score is :  0.8035695552825928
gfo  :  Self Care
gcph_brand  :  Tylenol
gcph_subbrand  :  Extra Strength
gcph_varient  :  Extra Strength - Core
python_score  :  0.8035695552825928
gcph_needstate  :  Pain Care
gcph_category  :  Pain & Sleep
gcph_subcategory  :  Internal Analgesics
gcph_segment  :  Adult Internal Analgesics
gcph_subsegment  :  Adult Internal Analgesics
Query is ==== > BAND-AID TOUGH CLOTH TAPE, 1 in x 10

gcph_brand  :  Band Aid
gcph_subbrand  :  Deco, Decorative, Decorated
gcph_varient  :  Deco, Decorative, Decorated
python_score  :  0.6982159614562988
gcph_needstate  :  Wound Care
gcph_category  :  Wound Protection
gcph_subcategory  :  Adhesive Bandages
gcph_segment  :  Adhesive Bandages
gcph_subsegment  :  Adhesive Bandages
Query is ==== > BANDAGE BAND-AID SANSDOULEUR, 2 po x 5 vg bandaid
0
(2911, 0.6328638195991516)
Match in Brand Sheet is :  ['band', 'aid', 'brand', 'bandaid', 'baby', 'shark'] Score is :  0.6328638195991516
Match in Category sheet is :  ['band', 'aid', 'brand', 'bandaid', 'baby', 'shark'] Score is :  0.6328638195991516
gfo  :  Essential Health
gcph_brand  :  Band Aid
gcph_subbrand  :  Deco, Decorative, Decorated
gcph_varient  :  Deco, Decorative, Decorated
python_score  :  0.6328638195991516
gcph_needstate  :  Wound Care
gcph_category  :  Wound Protection
gcph_subcategory  :  Adhesive Bandages
gcph_segment  :  Adhesive Bandages
gcph_subsegment  :  Adhesive Bandages

gfo  :  Essential Health
gcph_brand  :  Band Aid
gcph_subbrand  :  Deco, Decorative, Decorated
gcph_varient  :  Deco, Decorative, Decorated
python_score  :  0.5763022899627686
gcph_needstate  :  Wound Care
gcph_category  :  Wound Protection
gcph_subcategory  :  Adhesive Bandages
gcph_segment  :  Adhesive Bandages
gcph_subsegment  :  Adhesive Bandages
Query is ==== > NICODERM Step 3- 7mg Patch nicoderm
0
(421, 0.7537104487419128)
Match in Brand Sheet is :  ['nicoderm', 'nicorette', 'patch'] Score is :  0.7537104487419128
Match in Category sheet is :  ['nicoderm', 'nicorette', 'patch'] Score is :  0.7537104487419128
gfo  :  Self Care
gcph_brand  :  Nicorette
gcph_subbrand  :  Nicorette
gcph_varient  :  Nicorette
python_score  :  0.7537104487419128
gcph_needstate  :  Smoking Cessation
gcph_category  :  Total Smoking Cessation
gcph_subcategory  :  Patch
gcph_segment  :  Patch
gcph_subsegment  :  Patch
Query is ==== > PETITES COMPRESSES DE GAZE BAND-AID PROTECTION COUSSINÉE, Emballage écono

...

0
(7196, 0.7425276637077332)
Match in Brand Sheet is :  ['polysporin', 'cracked', 'skin', 'ointment'] Score is :  0.7425276637077332
Match in Category sheet is :  ['polysporin', 'cracked', 'skin', 'ointment'] Score is :  0.7425276637077332
gfo  :  Essential Health
gcph_brand  :  Polysporin
gcph_subbrand  :  Eye/Ear Antibiotics
gcph_varient  :  Eye/Ear Antibiotics
python_score  :  0.7425276637077332
gcph_needstate  :  Wound Care
gcph_category  :  Wound Protection
gcph_subcategory  :  Antibiotic Ointments
gcph_segment  :  Antibiotic Ointments
gcph_subsegment  :  Antibiotic Ointments
Query is ==== > SUDAFED RHUME DE CERVEAU ET SINUS sudafed
0
(6896, 0.8624369502067566)
Match in Brand Sheet is :  ['sudafed', 'sinus', 'complete'] Score is :  0.8624369502067566
Match in Category sheet is :  ['sudafed', 'sinus', 'complete'] Score is :  0.8624369502067566
gfo  :  Self Care
gcph_brand  :  Sudafed
gcph_subbrand  :  Head Decongestation
gcph_varient  :  Head Decongestation Core
python_score  :  0.

Match in Category sheet is :  ['listerine'] Score is :  1.0
gfo  :  Essential Health
gcph_brand  :  Listerine
gcph_subbrand  :  Cool Mint
gcph_varient  :  Cool Mint - Core
python_score  :  1.0
gcph_needstate  :  Oral Care
gcph_category  :  Oral Care
gcph_subcategory  :  Mouthwash
gcph_segment  :  Mouthwash
gcph_subsegment  :  Mouthwash
Query is ==== > CRÈME À MAINS AVEENO SOIN DE L’ECZÉMA aveeno2
0
(6426, 0.6683729887008667)
Match in Brand Sheet is :  ['aveeno', 'ave', 'hairspray'] Score is :  0.6683729887008667
Match in Category sheet is :  ['aveeno', 'ave', 'hairspray'] Score is :  0.6683729887008667
gfo  :  Skin Health
gcph_brand  :  Aveeno
gcph_subbrand  :  Aveeno - All Other
gcph_varient  :  Aveeno - All Other
python_score  :  0.6683729887008667
gcph_needstate  :  Hair Care
gcph_category  :  Hair Care
gcph_subcategory  :  Other Hair Care
gcph_segment  :  Hair Treatments & Styling
gcph_subsegment  :  Hair Treatments & Styling
Query is ==== > For Children Dry Cough Syrup benylin
0
(

0
(4136, 0.9453706741333008)
Match in Brand Sheet is :  ['aveeno', 'ave', 'eczema', 'care', 'moisturizing', 'cream'] Score is :  0.9453706741333008
Match in Category sheet is :  ['aveeno', 'ave', 'eczema', 'care', 'moisturizing', 'cream'] Score is :  0.9453706741333008
gfo  :  Skin Health
gcph_brand  :  Aveeno
gcph_subbrand  :  Eczema Therapy/ Dermexa
gcph_varient  :  Eczema Therapy/ Dermexa
python_score  :  0.9453706741333008
gcph_needstate  :  Body Care
gcph_category  :  Hand Body & Foot Moisturising
gcph_subcategory  :  Body Moisturising
gcph_segment  :  Adult Body Moisturising
gcph_subsegment  :  Adult Body Moisturising
Query is ==== > 1-DAY ACUVUEMD TruEyeMD acuvue
0
(1433, 0.65157550573349)
Match in Brand Sheet is :  ['nicorette', 'floorstand', 'day'] Score is :  0.65157550573349
Match in Category sheet is :  ['nicorette', 'floorstand', 'day'] Score is :  0.65157550573349
gfo  :  Self Care
gcph_brand  :  Nicorette
gcph_subbrand  :  Nicorette
gcph_varient  :  Nicorette
python_scor

gcph_category  :  Oral Care
gcph_subcategory  :  Mouthwash
gcph_segment  :  Mouthwash
gcph_subsegment  :  Mouthwash
Query is ==== > BAND-AID HURT-FREE Antiseptic Wash, 177 mL bandaid
0
(4017, 0.859127402305603)
Match in Brand Sheet is :  ['band', 'aid', 'brand', 'hurt', 'free', 'antiseptic', 'wash'] Score is :  0.859127402305603
Match in Category sheet is :  ['band', 'aid', 'brand', 'hurt', 'free', 'antiseptic', 'wash'] Score is :  0.859127402305603
gfo  :  Essential Health
gcph_brand  :  Band Aid
gcph_subbrand  :  Band Aid - All Other
gcph_varient  :  Antiseptic Washes / Sprays
python_score  :  0.859127402305603
gcph_needstate  :  Wound Care
gcph_category  :  Wound Protection
gcph_subcategory  :  Adhesive Bandages
gcph_segment  :  Adhesive Bandages
gcph_subsegment  :  Adhesive Bandages
Query is ==== > Pansements adhésifs assortis BAND-AID Disney Pixar Incredibles 2, 20 u. bandaid
0
(1264, 0.8450849056243896)
Match in Brand Sheet is :  ['band', 'aid', 'brand', 'bandaid', 'disney', 'emo

gcph_subbrand  :  MS Cold + Decon/Sinus
gcph_varient  :  MS Cold + Decon/Sinus
python_score  :  0.638503909111023
gcph_needstate  :  Cold & Cough
gcph_category  :  Cold & Cough
gcph_subcategory  :  CCFS
gcph_segment  :  Cough
gcph_subsegment  :  Cough
Query is ==== > BAND-AID FLEXIBLE ROLLED GAUZE SMALL, 2 in x 5 yds, 1 Sterile Roll  bandaid
0
(3654, 0.6347019076347351)
Match in Brand Sheet is :  ['johnson', 'rolled', 'gauze', 'yds'] Score is :  0.6347019076347351
Match in Category sheet is :  ['johnson', 'rolled', 'gauze', 'yds'] Score is :  0.6347019076347351
gfo  :  Essential Health
gcph_brand  :  Band Aid
gcph_subbrand  :  Band Aid First Aid
gcph_varient  :  Covers - Rolled Gauze
python_score  :  0.6347019076347351
gcph_needstate  :  Wound Care
gcph_category  :  Wound Protection
gcph_subcategory  :  First Aid
gcph_segment  :  First Aid
gcph_subsegment  :  First Aid
Query is ==== > ACUVUE OASYS Multifocal for PRESBYOPIA (2-Week) Contact Lenses acuvue
0
(7057, 0.470760703086853)
Matc

In [12]:
df_brand.iloc[7193]

tamr_id                                                                                    -2.53e+18
origin_source_name                                                                allbrands_skus.csv
origin_entity_id                                                                        CA-100037204
manualClassificationId                                                                           NaN
manualClassificationPath                                                                         NaN
suggestedClassificationId                                                                     153928
suggestedClassificationPath                        Essential Health|Band Aid|Skin-Flex/ Katiteki ...
suggestedClassificationTierConfidences             0.993498993140936|0.9923217959933007|0.9253669...
suggestedClassificationConfidence                                                           0.925055
suggestedClassificationIdAboveThreshold                                                    

In [ ]:
# import json
# file = open('data_from_python_us.json','w')
# file.write(json.dumps(final_output))

In [ ]:
from gensim.similarities import WmdSimilarity
tfidf = WmdSimilarity(common_texts, model)

query = ['trees']
sims = index[query]

# Please scroll above output

# Advantages :
- No paid tools only open source NLP model.
- Can improve performance with more feature engineering
- Model Build time less than 2 min as compared to TAMR 20mins
- Can predict taxonomy/hierarchy for all jnj products
- No training required as we are just finding similar text in brand and category sheet as compared to classfying things in TAMR.
- Can take into consideration/lookpup global data almost 2Lakh record as compared to 2000records of input file for tamr.

# Disadvantages : 
- You say ...
    